#### Sunday, December 17, 2023

[Tutorial: How to Use Pipelines](https://haystack.deepset.ai/tutorials/11_pipelines)

This all runs.

In [1]:
# only target the 4090 ...
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

# Tutorial: How to Use Pipelines

In this tutorial, you will learn how the `Pipeline` connects the different components in Haystack. Whether you are using a Reader, Summarizer
or Retriever (or 2), the `Pipeline` class will help you build a Directed Acyclic Graph (DAG) that
determines how to route the output of one component into the input of another.



## Preparing the Colab Environment

- [Enable GPU Runtime](https://docs.haystack.deepset.ai/docs/enabling-gpu-acceleration#enabling-the-gpu-in-colab)


## Installing Haystack

To start, let's install the latest release of Haystack with `pip`:

In [ ]:
# %%bash

# pip install --upgrade pip
# pip install farm-haystack[colab,inference]

### Enabling Telemetry
Knowing you're using this tutorial helps us decide where to invest our efforts to build a better product but you can always opt out by commenting the following line. See [Telemetry](https://docs.haystack.deepset.ai/docs/telemetry) for more details.

In [ ]:
from haystack.telemetry import tutorial_running

tutorial_running(11)

## Logging

We configure how logging messages should be displayed and which log level should be used before importing Haystack.
Example log message:
INFO - haystack.utils.preprocessing -  Converting data/tutorial1/218_Olenna_Tyrell.txt
Default log level in basicConfig is WARNING so the explicit parameter is not necessary but can be changed easily:

In [2]:
import logging

logging.basicConfig(format="%(levelname)s - %(name)s -  %(message)s", level=logging.WARNING)
logging.getLogger("haystack").setLevel(logging.INFO)

## Initializing the DocumentStore

A DocumentStore stores the Documents that a system uses to retrieve or find answers to your questions. In this tutorial, you'll use the `InMemoryDocumentStore`.

Let's initialize the DocumentStore.

In [3]:
from haystack.document_stores import InMemoryDocumentStore

document_store = InMemoryDocumentStore(use_bm25=True)

INFO - haystack.modeling.utils -  Using devices: CUDA:0 - Number of GPUs: 1


## Fetching and Writing Documents

Let's fetch the txt files (in this case, pages from the Game of Thrones wiki) and prepare them so that they can be indexed into the DocumentStore:

In [4]:
from haystack.utils import fetch_archive_from_http, convert_files_to_docs, clean_wiki_text

# Download and prepare data - 517 Wikipedia articles for Game of Thrones
doc_dir = "data/tutorial11"
s3_url = "https://s3.eu-central-1.amazonaws.com/deepset.ai-farm-qa/datasets/documents/wiki_gameofthrones_txt11.zip"
fetch_archive_from_http(url=s3_url, output_dir=doc_dir)

# convert files to dicts containing documents that can be indexed to our datastore
got_docs = convert_files_to_docs(dir_path=doc_dir, clean_func=clean_wiki_text, split_paragraphs=True)
document_store.delete_documents()
document_store.write_documents(got_docs)

INFO - haystack.utils.import_utils -  Fetching from https://s3.eu-central-1.amazonaws.com/deepset.ai-farm-qa/datasets/documents/wiki_gameofthrones_txt11.zip to 'data/tutorial11'
INFO - haystack.utils.preprocessing -  Converting data/tutorial11/44_Daenerys_Targaryen.txt
INFO - haystack.utils.preprocessing -  Converting data/tutorial11/134_Game_of_Thrones__Season_6__soundtrack_.txt
INFO - haystack.utils.preprocessing -  Converting data/tutorial11/310_Mother_s_Mercy.txt
INFO - haystack.utils.preprocessing -  Converting data/tutorial11/224_The_Night_Lands.txt
INFO - haystack.utils.preprocessing -  Converting data/tutorial11/340_Roose_Bolton.txt
INFO - haystack.utils.preprocessing -  Converting data/tutorial11/23_Game_of_Thrones_Live_Concert_Experience.txt
INFO - haystack.utils.preprocessing -  Converting data/tutorial11/395_Game_of_Thrones__season_5_.txt
INFO - haystack.utils.preprocessing -  Converting data/tutorial11/154_Margaery_Tyrell.txt
INFO - haystack.utils.preprocessing -  Converti

In [5]:
from haystack.nodes import BM25Retriever, EmbeddingRetriever, FARMReader

bm25_retriever = BM25Retriever()

from haystack.pipelines import DocumentSearchPipeline
from haystack.utils import print_documents

p_retrieval = DocumentSearchPipeline(bm25_retriever)
# res = p_retrieval.run(query="Who is the father of Arya Stark?", params={"Retriever": {"top_k": 10}})
# print_documents(res, max_text_len=200)

## Initializing Core Components

Here we initialize the core components that we will be gluing together using the `Pipeline` class.
Initialize a `BM25Retriever`, an `EmbeddingRetriever`, and a `FARMReader`.
You can combine these components to create a classic Retriever-Reader pipeline that is designed
to perform Open Domain Question Answering.

In [6]:
from haystack.nodes import BM25Retriever, EmbeddingRetriever, FARMReader

# Initialize Sparse Retriever
bm25_retriever = BM25Retriever(document_store=document_store)

# Initialize embedding Retriever
embedding_retriever = EmbeddingRetriever(
    document_store=document_store, embedding_model="sentence-transformers/multi-qa-mpnet-base-dot-v1"
)
document_store.update_embeddings(embedding_retriever, update_existing_embeddings=False)

# Initialize Reader
reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2")

# 6m 20.8s

INFO - haystack.modeling.utils -  Using devices: CUDA:0 - Number of GPUs: 1


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

INFO - haystack.nodes.retriever.dense -  Init retriever using embeddings of model sentence-transformers/multi-qa-mpnet-base-dot-v1


.gitattributes:   0%|          | 0.00/737 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

/usr/local/lib/python3.8/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
INFO - haystack.document_stores.memory -  Updating embeddings for 0 docs ...
Updating Embedding:   0%|          | 0/2357 [00:00<?, ? docs/s]

Batches:   0%|          | 0/74 [00:00<?, ?it/s]

Documents Processed: 10000 docs [00:06, 1586.94 docs/s]         
INFO - haystack.modeling.utils -  Using devices: CUDA:0 - Number of GPUs: 1
INFO - haystack.modeling.utils -  Using devices: CUDA:0 - Number of GPUs: 1
INFO - haystack.modeling.model.language_model -   * LOADING MODEL: 'deepset/roberta-base-squad2' (Roberta)
INFO - haystack.modeling.model.language_model -  Auto-detected model language: english
INFO - haystack.modeling.model.language_model -  Loaded 'deepset/roberta-base-squad2' (Roberta model) from model hub.
INFO - haystack.modeling.utils -  Using devices: CUDA:0 - Number of GPUs: 1


In [7]:
!ls /root/.cache/huggingface/hub

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


models--HuggingFaceH4--zephyr-7b-beta
models--bert-base-uncased
models--deepset--roberta-base-squad2
models--google--flan-t5-large
models--meta-llama--Llama-2-13b-hf
models--sentence-transformers--all-MiniLM-L6-v2
models--sentence-transformers--multi-qa-mpnet-base-dot-v1
version.txt


In [ ]:
# docker cp c8324b70601d://root/.cache/huggingface/hub/models--sentence-transformers--multi-qa-mpnet-base-dot-v1 /home/rob/Data3/huggingface/transformers
# Successfully copied 6.14kB to /home/rob/Data3/huggingface/transformers

# ?? why is this so small?? where is the pytorch_model.bin ??

## Prebuilt Pipelines

Haystack features many prebuilt pipelines that cover common tasks. The most used one is `ExtractiveQAPipeline`.
Now, initialize the ExtractiveQAPipeline and run it with a query:

In [8]:
from haystack.pipelines import ExtractiveQAPipeline
from haystack.utils import print_answers

p_extractive_premade = ExtractiveQAPipeline(reader=reader, retriever=bm25_retriever)
res = p_extractive_premade.run(
    query="Who is the father of Arya Stark?", params={"Retriever": {"top_k": 10}, "Reader": {"top_k": 5}}
)
print_answers(res, details="minimum")

Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00,  7.34 Batches/s]

'Query: Who is the father of Arya Stark?'
'Answers:'
[   {   'answer': 'Eddard',
        'context': 's Nymeria after a legendary warrior queen. She travels '
                   "with her father, Eddard, to King's Landing when he is made "
                   'Hand of the King. Before she leaves,'},
    {   'answer': 'Ned',
        'context': '\n'
                   '====Season 1====\n'
                   'Arya accompanies her father Ned and her sister Sansa to '
                   "King's Landing. Before their departure, Arya's "
                   'half-brother Jon Snow gifts A'},
    {   'answer': 'Lord Eddard Stark',
        'context': 'ark daughters.\n'
                   'During the Tourney of the Hand to honour her father Lord '
                   'Eddard Stark, Sansa Stark is enchanted by the knights '
                   'performing in the event.'},
    {   'answer': 'Eddard Stark',
        'context': "arrested by Catelyn Stark. He then races to King's Landing "
                 

If you want to just do the retrieval step, you can use a DocumentSearchPipeline:

In [9]:
from haystack.pipelines import DocumentSearchPipeline
from haystack.utils import print_documents

p_retrieval = DocumentSearchPipeline(embedding_retriever)
res = p_retrieval.run(query="Who is the father of Arya Stark?", params={"Retriever": {"top_k": 10}})
print_documents(res, max_text_len=200)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]


Query: Who is the father of Arya Stark?

{   'content': '\n'
               '=== Background ===\n'
               'Arya is the third child and younger daughter of Eddard and '
               'Catelyn Stark and is nine years old at the beginning of the '
               'book series.  She has five siblings: an older brother Robb, '
               'a...',
    'name': '43_Arya_Stark.txt'}

{   'content': '\n'
               '===Arya Stark===\n'
               "'''Arya Stark''' portrayed by Maisie Williams. Arya Stark of "
               'House Stark is the younger daughter and third child of Lord '
               'Eddard and Catelyn Stark of Winterfell. Ever the tomboy, '
               'Arya...',
    'name': '349_List_of_Game_of_Thrones_characters.txt'}

{   'content': "'''Arya Stark''' is a fictional character in American author "
               "George R. R. Martin's ''A Song of Ice and Fire'' epic fantasy "
               'novel series.  She is a prominent point of view character in '

Haystack features prebuilt pipelines to do:
- document search (DocumentSearchPipeline),
- document search with summarization (SearchSummarizationPipeline)
- FAQ style QA (FAQPipeline)
- translated search (TranslationWrapperPipeline)
To find out more about these pipelines, have a look at our [documentation](https://docs.haystack.deepset.ai/docs/pipelines).


## Generating a Pipeline Diagram

With any Pipeline, whether prebuilt or custom constructed,
you can save a diagram showing how all the components are connected.

![image](https://github.com/deepset-ai/haystack/blob/main/docs/img/retriever-reader-pipeline.png?raw=true)

In [ ]:
# Uncomment the following to generate the images
# !apt install libgraphviz-dev
# !pip install pygraphviz

# p_extractive_premade.draw("pipeline_extractive_premade.png")
# p_retrieval.draw("pipeline_retrieval.png")

## Custom Extractive QA Pipeline

If you need a custom pipeline for your task or add extra nodes to a pipeline, you can create a pipeline from scratch instead of using a prebuilt one.

Now, try to rebuild the ExtractiveQAPipeline using the generic Pipeline class.
You can do this by adding the building blocks as shown in the graph.

In [10]:
from haystack.pipelines import Pipeline

# Custom built extractive QA pipeline
p_extractive = Pipeline()
p_extractive.add_node(component=bm25_retriever, name="Retriever", inputs=["Query"])
p_extractive.add_node(component=reader, name="Reader", inputs=["Retriever"])

# Now we can run it
res = p_extractive.run(query="Who is the father of Arya Stark?", params={"Retriever": {"top_k": 10}})
print_answers(res, details="minimum")

# Uncomment the following to generate the pipeline image
# p_extractive.draw("pipeline_extractive.png")

Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00, 19.64 Batches/s]

'Query: Who is the father of Arya Stark?'
'Answers:'
[   {   'answer': 'Eddard',
        'context': 's Nymeria after a legendary warrior queen. She travels '
                   "with her father, Eddard, to King's Landing when he is made "
                   'Hand of the King. Before she leaves,'},
    {   'answer': 'Ned',
        'context': '\n'
                   '====Season 1====\n'
                   'Arya accompanies her father Ned and her sister Sansa to '
                   "King's Landing. Before their departure, Arya's "
                   'half-brother Jon Snow gifts A'},
    {   'answer': 'Lord Eddard Stark',
        'context': 'ark daughters.\n'
                   'During the Tourney of the Hand to honour her father Lord '
                   'Eddard Stark, Sansa Stark is enchanted by the knights '
                   'performing in the event.'},
    {   'answer': 'Eddard Stark',
        'context': "arrested by Catelyn Stark. He then races to King's Landing "
                 

## Combining Retrievers Using a Custom Pipeline

Pipelines offer a very simple way to ensemble together different components.
In this example, you'll combine the power of an `EmbeddingRetriever`
with the keyword based `BM25Retriever`.
See our [documentation](https://docs.haystack.deepset.ai/docs/retriever#deeper-dive-vector-based-vs-keyword-based) to understand why
you might want to combine a dense and sparse retriever.

![image](https://github.com/deepset-ai/haystack/blob/main/docs/img/tutorial11_custompipelines_pipeline_ensemble.png?raw=true)

Here you'll use a `JoinDocuments` node to merge the predictions from each retriever:

In [11]:
from haystack.nodes import JoinDocuments

# Create ensembled pipeline
p_ensemble = Pipeline()
p_ensemble.add_node(component=bm25_retriever, name="BM25Retriever", inputs=["Query"])
p_ensemble.add_node(component=embedding_retriever, name="EmbeddingRetriever", inputs=["Query"])
p_ensemble.add_node(
    component=JoinDocuments(join_mode="concatenate"), name="JoinResults", inputs=["BM25Retriever", "EmbeddingRetriever"]
)
p_ensemble.add_node(component=reader, name="Reader", inputs=["JoinResults"])

# Uncomment the following to generate the pipeline image
# p_ensemble.draw("pipeline_ensemble.png")

# Run pipeline
res = p_ensemble.run(
    query="Who is the father of Arya Stark?", params={"EmbeddingRetriever": {"top_k": 5}, "BM25Retriever": {"top_k": 5}}
)
print_answers(res, details="minimum")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00, 25.09 Batches/s]

'Query: Who is the father of Arya Stark?'
'Answers:'
[   {   'answer': 'Eddard',
        'context': 's Nymeria after a legendary warrior queen. She travels '
                   "with her father, Eddard, to King's Landing when he is made "
                   'Hand of the King. Before she leaves,'},
    {   'answer': 'Eddard and Catelyn Stark',
        'context': 'tark ===\n'
                   'Arya Stark is the third child and younger daughter of '
                   'Eddard and Catelyn Stark. She serves as a POV character '
                   "for 33 chapters throughout ''A "},
    {   'answer': 'Lord Eddard Stark',
        'context': 'ark daughters.\n'
                   'During the Tourney of the Hand to honour her father Lord '
                   'Eddard Stark, Sansa Stark is enchanted by the knights '
                   'performing in the event.'},
    {   'answer': 'Lord Eddard Stark',
        'context': "Game of Thrones'', Arya is the third child and younger "
                  

## Custom Nodes

Nodes are relatively simple objects
and we encourage our users to design their own if they don't see one that fits their use case

The only requirements are:
- Create a class that inherits `BaseComponent`.
- Add a method run() to your class. Add the mandatory and optional arguments it needs to process. These arguments must be passed as input to the pipeline, inside `params`, or output by preceding nodes.
- Add processing logic inside the run() (e.g. reformatting the query).
- Return a tuple that contains your output data (for the next node)
and the name of the outgoing edge (by default "output_1" for nodes that have one output)
- Add a class attribute outgoing_edges = 1 that defines the number of output options from your node. You only need a higher number here if you have a decision node (see below).

Here is a template for a custom node:

In [12]:
from haystack import BaseComponent
from typing import Optional, List


class CustomNode(BaseComponent):
    outgoing_edges = 1

    def run(self, query: str, my_optional_param: Optional[int]):
        # process the inputs
        output = {"my_output": ...}
        return output, "output_1"

    def run_batch(self, queries: List[str], my_optional_param: Optional[int]):
        # process the inputs
        output = {"my_output": ...}
        return output, "output_1"

## Decision Nodes

Decision Nodes help you route your data so that only certain branches of your `Pipeline` are run.
One popular use case for such query classifiers is routing keyword queries to Elasticsearch and questions to EmbeddingRetriever + Reader.
With this approach you keep optimal speed and simplicity for keywords while going deep with transformers when it's most helpful.

![image](https://github.com/deepset-ai/haystack/blob/main/docs/img/tutorial11_decision_nodes_pipeline_classifier.png?raw=true)

Though this looks very similar to the ensembled pipeline shown above,
the key difference is that only one of the retrievers is run for each request.
By contrast both retrievers are always run in the ensembled approach.

Below, we define a very naive `QueryClassifier` and show how to use it:

In [13]:
class CustomQueryClassifier(BaseComponent):
    outgoing_edges = 2

    def run(self, query: str):
        if "?" in query:
            return {}, "output_2"
        else:
            return {}, "output_1"

    def run_batch(self, queries: List[str]):
        split = {"output_1": {"queries": []}, "output_2": {"queries": []}}
        for query in queries:
            if "?" in query:
                split["output_2"]["queries"].append(query)
            else:
                split["output_1"]["queries"].append(query)

        return split, "split"


# Here we build the pipeline
p_classifier = Pipeline()
p_classifier.add_node(component=CustomQueryClassifier(), name="QueryClassifier", inputs=["Query"])
p_classifier.add_node(component=bm25_retriever, name="BM25Retriever", inputs=["QueryClassifier.output_1"])
p_classifier.add_node(component=embedding_retriever, name="EmbeddingRetriever", inputs=["QueryClassifier.output_2"])
p_classifier.add_node(component=reader, name="QAReader", inputs=["BM25Retriever", "EmbeddingRetriever"])
# Uncomment the following to generate the pipeline image
# p_classifier.draw("pipeline_classifier.png")

# Run only the dense retriever on the full sentence query
res_1 = p_classifier.run(query="Who is the father of Arya Stark?")
print("Embedding Retriever Results" + "\n" + "=" * 15)
print_answers(res_1)

# Run only the sparse retriever on a keyword based query
res_2 = p_classifier.run(query="Arya Stark father")
print("BM25Retriever Results" + "\n" + "=" * 15)
print_answers(res_2)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00, 26.13 Batches/s]


Embedding Retriever Results
'Query: Who is the father of Arya Stark?'
'Answers:'
[   <Answer {'answer': 'Eddard and Catelyn Stark', 'type': 'extractive', 'score': 0.9058834314346313, 'context': "tark ===\nArya Stark is the third child and younger daughter of Eddard and Catelyn Stark. She serves as a POV character for 33 chapters throughout ''A ", 'offsets_in_document': [{'start': 74, 'end': 98}], 'offsets_in_context': [{'start': 63, 'end': 87}], 'document_ids': ['965789c741c68963042e85b6e7b89757'], 'meta': {'name': '30_List_of_A_Song_of_Ice_and_Fire_characters.txt'}}>,
    <Answer {'answer': 'Lord Eddard Stark', 'type': 'extractive', 'score': 0.8793495893478394, 'context': "Game of Thrones'', Arya is the third child and younger daughter of Lord Eddard Stark and his wife Lady Catelyn Stark.  She is tomboyish, headstrong, f", 'offsets_in_document': [{'start': 419, 'end': 436}], 'offsets_in_context': [{'start': 67, 'end': 84}], 'document_ids': ['2ee56bdd46dfd30b23f91bcc046456a4'], 'meta':

Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00, 32.07 Batches/s]

BM25Retriever Results
'Query: Arya Stark father'
'Answers:'
[   <Answer {'answer': 'Eddard', 'type': 'extractive', 'score': 0.9085890054702759, 'context': "s Nymeria after a legendary warrior queen. She travels with her father, Eddard, to King's Landing when he is made Hand of the King. Before she leaves,", 'offsets_in_document': [{'start': 147, 'end': 153}], 'offsets_in_context': [{'start': 72, 'end': 78}], 'document_ids': ['ba2a8e87ddd95e380bec55983ee7d55f'], 'meta': {'name': '43_Arya_Stark.txt'}}>,
    <Answer {'answer': 'Ned', 'type': 'extractive', 'score': 0.7877888679504395, 'context': "\n====Season 1====\nArya accompanies her father Ned and her sister Sansa to King's Landing. Before their departure, Arya's half-brother Jon Snow gifts A", 'offsets_in_document': [{'start': 46, 'end': 49}], 'offsets_in_context': [{'start': 46, 'end': 49}], 'document_ids': ['180c2a6b36369712b361a80842e79356'], 'meta': {'name': '43_Arya_Stark.txt'}}>,
    <Answer {'answer': 'Lord Eddard Stark', 'type

## Evaluation Nodes

We have also designed a set of nodes that can be used to evaluate the performance of a system.
Have a look at our [tutorial](https://haystack.deepset.ai/tutorials/05_evaluation) to get hands on with the code and learn more about Evaluation Nodes!

## Debugging Pipelines

You can print out debug information from nodes in your pipelines in a few different ways.

In [14]:
# 1) You can set the `debug` attribute of a given node.
bm25_retriever.debug = True

# 2) You can provide `debug` as a parameter when running your pipeline
result = p_classifier.run(query="Who is the father of Arya Stark?", params={"BM25Retriever": {"debug": True}})

# 3) You can provide the `debug` paramter to all nodes in your pipeline
result = p_classifier.run(query="Who is the father of Arya Stark?", params={"debug": True})

result["_debug"]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00, 25.96 Batches/s]


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00, 26.18 Batches/s]


{'Query': {'input': {'debug': True}, 'output': {}, 'exec_time_ms': 0.04},
 'QueryClassifier': {'input': {'query': 'Who is the father of Arya Stark?',
   'debug': True},
  'output': {},
  'exec_time_ms': 0.04},
 'EmbeddingRetriever': {'input': {'root_node': 'Query',
   'query': 'Who is the father of Arya Stark?',
   'debug': True},
  'output': {'documents': [<Document: {'content': '\n=== Background ===\nArya is the third child and younger daughter of Eddard and Catelyn Stark and is nine years old at the beginning of the book series.  She has five siblings: an older brother Robb, an older sister Sansa, two younger brothers Bran and Rickon, and an older illegitimate half-brother, Jon Snow.', 'content_type': 'text', 'score': 0.5609863183876689, 'meta': {'name': '43_Arya_Stark.txt'}, 'id_hash_keys': ['content'], 'embedding': None, 'id': 'd7a98cb66f592540fa7de20bf46a5e64'}>,
    <Document: {'content': "\n===Arya Stark===\n'''Arya Stark''' portrayed by Maisie Williams. Arya Stark of House Sta

## Conclusion

The possibilities are endless with the `Pipeline` class and we hope that this tutorial will inspire you
to build custom pipelines that really work for your use case!